In [ ]:
import requests
import json
import csv
import time

def fetch_fda_label_efficacy(drug_name):
    url = "https://api.fda.gov/drug/label.json"
    params = {
        "search": f'openfda.generic_name:"{drug_name}"',
        "limit": 1
    }

    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        if "results" not in data:
            return {"drug_name": drug_name, "error": "No results found"}

        result = data["results"][0]
        return {
            "drug_name": drug_name,
            "indications_and_usage": result.get("indications_and_usage", ["Not available"])[0][:1000],
            "clinical_studies": result.get("clinical_studies", ["Not available"])[0][:1000],
            "dosage_and_administration": result.get("dosage_and_administration", ["Not available"])[0][:1000],
            "warnings_and_cautions": result.get("warnings_and_cautions", ["Not available"])[0][:1000],
            "contraindications": result.get("contraindications", ["Not available"])[0][:1000],
            "pregnancy": result.get("pregnancy", ["Not available"])[0][:1000],
            "pediatric_use": result.get("pediatric_use", ["Not available"])[0][:1000],
            "geriatric_use": result.get("geriatric_use", ["Not available"])[0][:1000],
        }

    except Exception as e:
        return {"drug_name": drug_name, "error": str(e)}

# 🔢 List of 100 common generic drugs
drug_list = [
    "acetaminophen", "albuterol", "alprazolam", "amitriptyline", "amlodipine", "amoxicillin", "atorvastatin",
    "azithromycin", "benazepril", "bisoprolol", "bupropion", "buspirone", "carvedilol", "cephalexin", "ciprofloxacin",
    "citalopram", "clindamycin", "clonazepam", "clonidine", "clopidogrel", "cyclobenzaprine", "dexamethasone",
    "diazepam", "diclofenac", "dicyclomine", "digoxin", "diltiazem", "diphenhydramine", "divalproex", "donepezil",
    "doxycycline", "enalapril", "escitalopram", "esomeprazole", "famotidine", "fentanyl", "ferrous sulfate",
    "fluconazole", "fluoxetine", "fluticasone", "furosemide", "gabapentin", "glipizide", "glyburide", "guaifenesin",
    "haloperidol", "hydralazine", "hydrochlorothiazide", "hydrocodone", "hydroxyzine", "ibuprofen", "insulin glargine",
    "ipratropium", "irbesartan", "isosorbide", "ketoconazole", "lamotrigine", "lansoprazole", "lisinopril",
    "loratadine", "lorazepam", "losartan", "lovastatin", "meloxicam", "metformin", "methocarbamol", "methotrexate",
    "methylprednisolone", "metoprolol", "metronidazole", "montelukast", "morphine", "naproxen", "nitrofurantoin",
    "nitroglycerin", "nortriptyline", "omeprazole", "ondansetron", "oxycodone", "pantoprazole", "paroxetine",
    "penicillin", "phenobarbital", "phenytoin", "pioglitazone", "pravastatin", "prednisone", "pregabalin",
    "propranolol", "quetiapine", "ranitidine", "risperidone", "rosuvastatin", "sertraline", "simvastatin",
    "sulfamethoxazole", "sumatriptan", "tamsulosin", "temazepam", "topiramate", "tramadol", "trazodone", "valacyclovir",
    "valsartan", "venlafaxine", "warfarin", "zolpidem"
]

results = []
for i, drug in enumerate(drug_list):
    print(f"[{i+1}/{len(drug_list)}] Fetching: {drug}")
    data = fetch_fda_label_efficacy(drug)
    results.append(data)
    time.sleep(1)  # Respect OpenFDA rate limits

# 💾 Save as JSON
with open("drug_efficacy_data.json", "w") as f:
    json.dump(results, f, indent=2)

# 💾 Save as CSV
with open("drug_efficacy_data.csv", "w", newline='', encoding='utf-8') as f:
    fieldnames = ["drug_name", "indications_and_usage", "clinical_studies", "dosage_and_administration","warnings_and_cautions",
    "contraindications","pregnancy","pediatric_use","geriatric_use","error"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for row in results:
        writer.writerow({
            "drug_name": row.get("drug_name"),
            "indications_and_usage": row.get("indications_and_usage", ""),
            "clinical_studies": row.get("clinical_studies", ""),
            "dosage_and_administration": row.get("dosage_and_administration", ""),
            "warnings_and_cautions": row.get("warnings_and_cautions", ""),
            "contraindications": row.get("contraindications", ""),
            "pregnancy": row.get("pregnancy", ""),
            "pediatric_use": row.get("pediatric_use", ""),
            "geriatric_use": row.get("geriatric_use", ""),
            "error": row.get("error", "")
        })

print("✅ Done! Data saved to drug_efficacy_data.json and drug_efficacy_data.csv")


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")
os.environ["LANGSMITH_TRACING_V2"] = os.getenv("LANGSMITH_TRACING_V2")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")

In [ ]:
##Test Groq Connection
from langchain_groq import ChatGroq
llm = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2)
    


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful translator that translates from {language1} to {language2}.",),
        ("human","{input}"),



    ]
)

In [ ]:
chain = prompt|llm
chain.invoke(
    {
        "language1" : "English",
        "language2" : "Dutch",
        "input" : "This is trnslation using llm.",

    }
)

Creating a Document loader to load the product label json Data

In [ ]:
from langchain_community.document_loaders import JSONLoader
loader=JSONLoader(file_path="./drug-label-0013-of-0013.json",
                jq_schema=".results[]",
                text_content=False
                )

In [ ]:
document = loader.load()

Verifying Sample Data

In [ ]:
document[0]

In [ ]:
print(document[0].metadata)

Now as a next step , splitting the document into multiple  pages using text splitter

In [63]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
pages=text_splitter.split_documents(document)

In [64]:
len(pages)

476602

In [ ]:
os.environ["NOMIC_API_KEY"] = os.getenv("NOMIC_API_KEY")

In [65]:
from langchain_nomic import NomicEmbeddings
embeddings=NomicEmbeddings(model="nomic-embed-text-v1.5",dimensionality=384)

In [ ]:
from pinecone import Pinecone


In [ ]:
os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY")

In [ ]:
pc=Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

In [ ]:
from pinecone import ServerlessSpec

In [66]:
index_name ="medicalqueryasst"

if  pc.has_index(index_name):
    pc.delete_index(index_name)
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws",region="us-east-1") 
)

In [67]:
index=pc.Index(index_name)

In [68]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(pages))]

In [ ]:
from langchain_pinecone import PineconeVectorStore

In [69]:
vector_store=PineconeVectorStore(index=index,embedding=embeddings)

In [70]:
vector_store.add_documents(documents=pages, ids=uuids)

Exception: (429, 'Please try your request again later')